## A very simple kind of adapter

from Chapter 4 of the book: <a href=http://0-proquestcombo.safaribooksonline.com.sally.sandiego.edu/book/programming/python/9781783989324/mastering-python-design-patterns/ch04_html>Mastering Python Design Patterns</a>

Imagine that we have an application in which we run

    for obj in objects:
          obj.execute()
          
Of course, each obj in the sequence has an `execute` method.          

In [1]:
# adapter1.py
class Program:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'the {} program'.format(self.name)

    def execute(self):
        return 'executes a program'

class Computer:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'a {} computer'.format(self.name)

    def execute(self):
        return 'Using {}'.format(str(self))


if __name__ == "__main__":

    objects = [Computer('Asus'), Computer('HP'), Program('Excel'), Program('Emacs')]

    for i in objects:
        print('{} {}'.format(str(i), i.execute()))

a Asus computer Using a Asus computer
a HP computer Using a HP computer
the Excel program executes a program
the Emacs program executes a program


As our application evolves and expands, imagine that we would like to include in this loop objects that do not have an `execute` method.  I.e., we wish our loop to work with objects with **different interface** that do not work with our loop out of box.  In this simple example, we assume that such new objects have similarly functioning methods, just that they are not named `execute`.  The following are two such example objects:

In [2]:
# external.py
class Synthesizer:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'the {} synthesizer'.format(self.name)

    def play(self):
        return str(self)

class Human:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return '{} the human'.format(self.name)

    def speak(self):
        return str(self)

You can see that simply including synthesizers and humans in the loop will raise exception.

A naive way to integrate such new objects into the loop is to create an adapter for each such new object.  This is too much work, boring and not scalable.  See below.

In [3]:

#from external import Synthesizer, Human
class Program:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'the {} program'.format(self.name)

    def execute(self):
        return 'executes {}'.format(self.name)

class Computer:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def execute(self):
        return 'Using a {}'.format(self.name)

class HumanAdapter:

    def __init__(self, human):
        self.human = human

    def execute(self):
        return self.human.speak()

    def __str__(self):
        return str(self.human)

class SynthesizerAdapter:

    def __init__(self, synthesizer):
        self.synthesizer = synthesizer

    def execute(self):
        return self.synthesizer.play()

    def __str__(self):
        return str(self.synthesizer)
    
if __name__ == "__main__":
    objects = [Computer('Asus'), Computer('HP'), Program('Excel'), Program('Emacs')]

    synth = Synthesizer('moog')
    s = SynthesizerAdapter(synth)
    objects.append(s)

    bob = Human('Bob')
    bob = HumanAdapter(bob)
    objects.append(bob)
    
    for i in objects:
        print(i.execute())

Using a Asus
Using a HP
executes Excel
executes Emacs
the moog synthesizer
Bob the human


Just as in the case of **wrapper as proxy**, we create one wrapper class that is able to proxy any kind of objects, so now we are to provide a _universal_ adapter for all objects. 

In [4]:
#from external import Synthesizer, Human

class Program:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'the {} program'.format(self.name)

    def execute(self):
        return 'executes {}'.format(self.name)

class Computer:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def execute(self):
        return 'Using a {}'.format(self.name)

"""The Adapter only wraps the adaptee.  The obligation of the
interface adaptation rests with the client during instantiation.
That is, when instantiating, the client must provide the mapping:

   method app uses (e.g., execute)-> wrapped object's method

It is with this mapping that the adapter is able to connect the
new interface with the old.
"""

class Adapter:
    def __init__(self, obj, adapted_methods):
    
        self._obj = obj          # wraps the object 
        self.__dict__.update(adapted_methods) # provides the execute method

    def __str__(self):
        return str(self._obj)

if __name__ == "__main__":
    objects = [Computer('Asus'), Computer('HP'), Program('Excel'), Program('Emacs')]

    synth = Synthesizer('moog')
    s = Adapter(synth, dict(execute=synth.play))
    objects.append(s)

    bob = Human('Bob')
    bob = Adapter(bob, dict(execute=bob.speak))
    # bob = Adapter(bob, {'execute' : bob.speak})    
    objects.append(bob)
    print(vars(bob))
    
    for obj in objects:
        print(obj.execute())

{'execute': <bound method Human.speak of <__main__.Human object at 0x1064a8a90>>, '_obj': <__main__.Human object at 0x1064a8a90>}
Using a Asus
Using a HP
executes Excel
executes Emacs
the moog synthesizer
Bob the human


Don't be fooled into thinking that you are reading some *totally new kind of code*.  You have seen this code in the second week of the semester, when we worked on HTMLgen, using a different syntax:  

In [5]:
#from external import Synthesizer, Human

class Program:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'the {} program'.format(self.name)

    def execute(self):
        return 'executes {}'.format(self.name)

class Computer:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def execute(self):
        return 'Using a {}'.format(self.name)

class Adapter:
    def __init__(self, obj, **kw):
        """ It is more natural to use keyword arguments """
        self._obj = obj
        for method in kw:
            setattr(self, method, kw[method])
        
    def __str__(self):
        return str(self._obj)

if __name__ == "__main__":
    objects = [Computer('Asus'), Computer('HP'), Program('Excel'), Program('Emacs')]

    synth = Synthesizer('moog')
    s = Adapter(synth, execute=synth.play)
    objects.append(s)

    bob = Human('Bob')
    bob = Adapter(bob, execute=bob.speak)
    objects.append(bob)

    for i in objects:
        print(i.execute())


Using a Asus
Using a HP
executes Excel
executes Emacs
the moog synthesizer
Bob the human


Note that this code is similar to `ImmutableSet.__init__`:

In [6]:
class ImmutableSet:

    def __init__(self, aList=[]):
        s = ListSet(aList)
        self.members = s.members
        self.intersection = s.intersection

In the eyes of a Java programmer, `__init__` is strange:  it defines no instance variables, and it defines no instance methods!